In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_df = pd.read_csv(file_to_load)

In [2]:
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Player Count
# Display total number of players
playerlist = purchase_df["SN"].unique() # in order to get the correct count, we will have to exclude the SN duplicates

# use "Total Players" as key, and len() of playerlist as value to create a dictionary and display
total_player_dict = {"Total Players": [len(playerlist)]}
total_player = pd.DataFrame(total_player_dict)
total_player

,Total Players
0,576


In [4]:
# Purchasing Analysis (Total)
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

# collect all the needed values for creating the "Purchasing Analysis (Total)" summary table
unique_items = purchase_df["Item ID"].nunique()
average_price = purchase_df["Price"].mean()
purchase_count = purchase_df["Price"].count()
total_revenue = purchase_df["Price"].sum()

# put all the element together, keys and values
purchasing_analysis = pd.DataFrame([{"Number of Unique Items":unique_items,
                                     "Average Price":average_price,
                                     "Number of Purchases":purchase_count,
                                     "Total Revenue":total_revenue
                                     }])


# then format using .map()
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${:.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)


purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
# Gender Demographics
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed


no_duplicate_SN = purchase_df.copy() # I don't want to mess up the original raw data frame
# create a new one but drop all the duplicated SNs while keeping corresponding values of other columns
no_duplicate_SN.drop_duplicates(subset="SN", keep='first', inplace=True) # drop duplicated SN but keep the first one

# collect all the needed values for creating the "Gender Demographics" summary table
gender_count = no_duplicate_SN["Gender"].value_counts() # Total Count
gender_percentage = gender_count / len(playerlist) * 100 # Percentage of Players

# put all the element together, keys and values
gender_demographics = pd.DataFrame({"Gender" : ["Male", "Female", "Other / Non-Disclosed"], 
                                    "Total Count":gender_count, 
                                    "Percentage of Players":gender_percentage})

# then format using .map()
gender_demographics["Percentage of Players"] = gender_demographics["Percentage of Players"].map("{:.2f}%".format)

# create a header "Gender" for the index
gender_demographics = gender_demographics.set_index(["Gender"])

gender_demographics

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
# Purchasing Analysis (Gender)
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
# Create a summary data frame to hold the results
# Optional: give the displayed data cleaner formatting
# Display the summary data frame

# this time I will need the duplicated SNs to get the correct purchase count
# use .groupby() to get calculations based on gender
gender_analysis = purchase_df.groupby(["Gender"])

# collect all the needed values for creating the "Purchasing Analysis (Gender)" summary table
gender_purchase_count = purchase_df["Gender"].value_counts() # Purchase Count
total_purchase_by_gender = gender_analysis["Price"].sum() # Total Purchase Value
gender_average = gender_analysis["Price"].mean() # Average Purchase Price
averge_per_person_gender = total_purchase_by_gender/gender_count # Avg Purchase Total per Person

# put all the element together, keys and values
purchasing_analysis_gender = pd.DataFrame({ "Gender" : ["Male", "Female", "Other / Non-Disclosed"],
                                            "Purchase Count" : gender_purchase_count, 
                                            "Average Purchase Price" : gender_average,
                                            "Total Purchase Value" : total_purchase_by_gender,
                                            "Avg Purchase Total per Person" : averge_per_person_gender
                                            })

# then format using .map()
purchasing_analysis_gender["Average Purchase Price"] = purchasing_analysis_gender["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_gender["Total Purchase Value"] = purchasing_analysis_gender["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis_gender["Avg Purchase Total per Person"] = purchasing_analysis_gender["Avg Purchase Total per Person"].map("${:.2f}".format)

# create a header "Gender" for the index
purchasing_analysis_gender = purchasing_analysis_gender.set_index(["Gender"])

purchasing_analysis_gender


# Avg total purchase per person = total purchase value / gender count (unique)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
Gender,,,,
Male,113,$3.20,$361.94,$4.47
Female,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Age Demographics
# Establish bins for ages
    # Use Max and Min to help define the bins
    # max_age = max(purchase_df["Age"])
    # min_age = min(purchase_df["Age"])
    # print(f"Max Age: {max_age}, Min Age: {min_age}"")
# Categorize the existing players using the age bins. Hint: use pd.cut()
# Calculate the numbers and percentages by age group
# Create a summary data frame to hold the results
# Optional: round the percentage column to two decimal points
# Display Age Demographics Table

# use bin[] to create age range
# Remember to use no duplicate
age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 50]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
no_duplicate_SN["Age Range"] = pd.cut(no_duplicate_SN["Age"], age_bins, labels=group_names, include_lowest=True) 

# collect all the needed values for creating the "Age Demographics" summary table
age_purchase_count = no_duplicate_SN["Age Range"].value_counts() # Total Count
age_percentage = age_purchase_count / len(playerlist) * 100 # Percentage of Players

# put all the element together, keys and values
age_demographics = pd.DataFrame({"Total Count" : age_purchase_count, "Percentage of Players" : age_percentage})

# then format using .map()
age_demographics["Percentage of Players"] = age_demographics["Percentage of Players"].map("{:.2f}%".format)

# use .sort_index() to dispay age range in desired order
age_demographics.sort_index() 

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
# Purchasing Analysis (Age)
# Bin the purchase_data data frame by age
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
# Create a summary data frame to hold the results
# Optional: give the displayed data cleaner formatting
# Display the summary data frame

age_bined_purchase_df = purchase_df.copy() # I don't want to mess with original data frame
age_bined_purchase_df["Age Range"] = pd.cut(age_bined_purchase_df["Age"], age_bins, labels=group_names, include_lowest=True)

# create one grouped by age range
age_analysis = age_bined_purchase_df.groupby(["Age Range"])


# collect all the needed values for creating the "Purchasing Analysis (Age)" summary table

# age_purchase_count = age_bined_purchase_df["Age Range"].value_counts() # Purchase Count
# ^^^^^^^^^^^^^^^^^ made a mistake of redefining age_purchase_count when I already did
# this messed up my calculations for Avg Total Purchase per Person

total_purchase_by_age = age_analysis["Price"].sum() # Total Purchase Value
age_average = age_analysis["Price"].mean() # Average Purchase Price
averge_per_person_age = total_purchase_by_age / age_purchase_count # Avg Total Purchase per Person


# unique_age_count = age_analysis["SN"].nunique() # Unique age count no duplicates
# averge_per_person_age = total_purchase_by_age / unique_age_count # Avg Total Purchase per Person
# ^^^^^^^^^^^^^^^^^ just to record what I did when I messed up with age_purchase_count
# I was able to get away with using .nunique() to get the correct calculation of Avg Total Purchase per Person

# put all the element together, keys and values
purchasing_analysis_age = pd.DataFrame({ "Purchase Count" : age_purchase_count, 
                                         "Average Purchase Price" : age_average,
                                         "Total Purchase Value" : total_purchase_by_age,
                                         "Avg Purchase Total per Person" : averge_per_person_age
                                         })

# then format using .map()
purchasing_analysis_age["Average Purchase Price"] = purchasing_analysis_age["Average Purchase Price"].map("${:.2f}".format)
purchasing_analysis_age["Total Purchase Value"] = purchasing_analysis_age["Total Purchase Value"].map("${:.2f}".format)
purchasing_analysis_age["Avg Purchase Total per Person"] = purchasing_analysis_age["Avg Purchase Total per Person"].map("${:.2f}".format)

purchasing_analysis_age



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total per Person
<10,17,$3.35,$77.13,$4.54
10-14,22,$2.96,$82.78,$3.76
15-19,107,$3.04,$412.89,$3.86
20-24,258,$3.05,$1114.06,$4.32
25-29,77,$2.90,$293.00,$3.81
30-34,52,$2.93,$214.00,$4.12
35-39,31,$3.60,$147.67,$4.76
40+,12,$2.94,$38.24,$3.19


In [9]:
# Top Spenders
# Run basic calculations to obtain the results in the table below
# Create a summary data frame to hold the results
# Sort the total purchase value column in descending order
# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame

# group by SN
top_spender_analysis = purchase_df.groupby(["SN"])


# collect all the needed values for creating the "Top Spenders" summary table
top_purchase_count = top_spender_analysis["SN"].count() # Purchase Count
total_purchase_top = top_spender_analysis["Price"].sum() # Total Purchase Value
top_average = top_spender_analysis["Price"].mean() # Average Purchase Price

# put all the element together, keys and values
purchasing_analysis_top = pd.DataFrame({ "Purchase Count" : top_purchase_count, 
                                         "Average Purchase Price" : top_average,
                                         "Total Purchase Value" : total_purchase_top })

# sort the data frame to the desired order
sorted_purchasing_analysis_top = purchasing_analysis_top.sort_values("Total Purchase Value", ascending=False)

# then format using .map()
sorted_purchasing_analysis_top["Average Purchase Price"] = sorted_purchasing_analysis_top["Average Purchase Price"].map("${:.2f}".format)
sorted_purchasing_analysis_top["Total Purchase Value"] = sorted_purchasing_analysis_top["Total Purchase Value"].map("${:.2f}".format)


sorted_purchasing_analysis_top.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
# Most Popular Items
# Retrieve the Item ID, Item Name, and Item Price columns
# Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
# Create a summary data frame to hold the results
# Sort the purchase count column in descending order
# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame

# group by Item ID and Item Name to show both in index columns
popular_item_analysis = purchase_df.groupby(["Item ID", "Item Name"])

# collect all the needed values for creating the "Most Popular Items" summary table
popular_purchase_count = popular_item_analysis["Item ID"].count() # Purchase Count
total_purchase_popular = popular_item_analysis["Price"].sum() # Total Purchase Value
item_price_popular = popular_item_analysis["Price"].mean() # Item Price

# put all the element together, keys and values
purchasing_analysis_popular = pd.DataFrame({ "Purchase Count" : popular_purchase_count, 
                                             "Item Price" : item_price_popular,
                                             "Total Purchase Value" : total_purchase_popular})

# sort to the desired order
sorted_purchasing_analysis_popular = purchasing_analysis_popular.sort_values("Purchase Count", ascending=False)

# then format using .map()
sorted_purchasing_analysis_popular["Item Price"] = sorted_purchasing_analysis_popular["Item Price"].map("${:.2f}".format)
sorted_purchasing_analysis_popular["Total Purchase Value"] = sorted_purchasing_analysis_popular["Total Purchase Value"].map("${:.2f}".format)

# sorted_purchasing_analysis_popular.style.set_properties(**{'text-align': 'left'})
# ^^^^^^^^^^^^^ experimenting it to align the texts to the left like shown in instruction
# it didn't work, maybe a jupyter notebook display setting thnigy

sorted_purchasing_analysis_popular.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [11]:
# Most Profitable Items
# Sort the above table by total purchase value in descending order
# Optional: give the displayed data cleaner formatting
# Display a preview of the data frame

# sort the data frame
profitable_item_analysis = purchasing_analysis_popular.sort_values("Total Purchase Value", ascending=False)

# then format using .map()
profitable_item_analysis["Item Price"] = profitable_item_analysis["Item Price"].map("${:.2f}".format)
profitable_item_analysis["Total Purchase Value"] = profitable_item_analysis["Total Purchase Value"].map("${:.2f}".format)

profitable_item_analysis.head()



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
